In [ ]:
import os
import requests
import webbrowser
import http.server
import socketserver
from urllib.parse import urlparse, parse_qs
from dotenv import load_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REDIRECT_URI = os.getenv('REDIRECT_URI')


BASE_URL = 'https://www.strava.com/api/v3/'
# Strava API endpoints
AUTHORIZATION_URL = f'{BASE_URL}oauth/authorize'
TOKEN_URL = f'{BASE_URL}oauth/token'
ACTIVITIES_URL = f'{BASE_URL}activities'


def get_access_token(code):
    """takes auth token code, returns access token."""
    
    token_body = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'code': code,
        'grant_type': 'authorization_code'
    }

    token_response = requests.post(TOKEN_URL, data=token_body).json()
    print(token_response)
    return token_response.get('access_token')

def get_activities(access_token):
    """Returns activities, takes access_token."""
    
    params = {
        'per_page': 30
    }
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {access_token}'
    }
    activities_response = requests.get(ACTIVITIES_URL, headers=headers, params=params)
    print(activities_response)
    if activities_response.status_code == 200:
        return activities_response.json()
    else:
        print(f"Request failed. Status code {activities_response.status_code}")
        return None

def handle_oauth_callback(path):
    """Handles the OAuth2 callback using method get_acccess_token."""
    #retrieving params from url
    #https://stackoverflow.com/questions/5074803/retrieving-parameters-from-a-url
    parsed_url = urlparse(path)
    query_params = parse_qs(parsed_url.query)
    code = query_params.get('code', [None])[0]

    if code:
        access_token = get_access_token(code)
        if access_token:

            activities_data = get_activities(access_token)
            if activities_data:
                print('Recieved activity data')
                convert_and_save_activity(activities_data)
            return (200, 'Authorization successful.')
        else:
            return (400, 'Failed to exchange code for token.')
    else:
        return (400, 'Authorization failed.')
    

def convert_and_save_activity(activities_data):
    print(activities_data)
    df = pd.json_normalize(activities_data)
    csv_file_path = 'activities.csv'
    df.to_csv(csv_file_path, index=False)
    print(f"DataFrame saved to {csv_file_path}")
    return df

In [ ]:
#Redirect the user to the Strava authorization URL
authorization_url = f"{AUTHORIZATION_URL}?client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}&response_type=code&scope=activity:read"
print("Opening authorization URL in web browser...")
webbrowser.open(authorization_url)

# Step 2: Handle the callback using a simple HTTP server
class SimpleHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        status_code, message = handle_oauth_callback(self.path)
        self.send_response(status_code)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        self.wfile.write(message.encode('utf-8'))
# 
# Start the HTTP server to handle the callback
PORT = 5000
with socketserver.TCPServer(("", PORT), SimpleHTTPRequestHandler) as httpd:
    print(f"Serving on port {PORT}")
    httpd.handle_request()

In [ ]:
act  = pd.read_csv('activities.csv')
test = act[['start_date', 'distance']]


In [ ]:
test['start_date']= pd.to_datetime(act['start_date'])
test['start_date'] = test['start_date'].dt.date
test = test.sort_values(by=['start_date'], ascending=True)

In [ ]:
sns.barplot(data=test, x='start_date', y='distance')

plt.title('Bar Plot of Activity Distances')
plt.xlabel('Activity Name')
plt.ylabel('Distance (meters)')
plt.xticks(rotation=45)

plt.show()

In [ ]:
run = act[['start_date', 'average_speed', 'max_speed']]

run

In [ ]:
act.info()